In [1]:
print("hello")

hello


In [2]:
!export CUBLAS_WORKSPACE_CONFIG=:4096:8

In [3]:
import warnings
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '2'
# import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' 

In [4]:
import torch
# import clip
# !pip install git+https://github.com/openai/CLIP.git
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
device

'cuda'

In [6]:
#Fine tune clip

In [7]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-2p4j5nca
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-2p4j5nca
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [8]:
!dir


archive			   full_text_description.csv
archive.zip		   last_model1.pt
best_model_01.pt	   meme-project-clean-json
best_model.pt		   meme-project-raw
checkpoints		   output_images_byAll
clip-explainability.ipynb  output_images_label_01
corrected_code\ (1).ipynb  replacewords.ipynb
Fine_explain.ipynb	   Transformer-MM-Explainability


In [9]:
!git clone https://github.com/hila-chefer/Transformer-MM-Explainability

import os
os.chdir(f'./Transformer-MM-Explainability')

!pip install einops
!pip install ftfy
!pip install captum

fatal: destination path 'Transformer-MM-Explainability' already exists and is not an empty directory.


In [10]:
!pip install pytorch-lightning


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [10]:
import torch
import CLIP.clip as clip
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
from captum.attr import visualization

In [11]:
#@title Control context expansion (number of attention layers to consider)
#@title Number of layers for image Transformer
start_layer =  -1

#@title Number of layers for text Transformer
start_layer_text =  -1

In [12]:
def interpret(image, texts, model, device, start_layer=start_layer, start_layer_text=start_layer_text):
    batch_size = texts.shape[0]
   # print('batch_size',batch_size)
    images = image.repeat(batch_size, 1, 1, 1)
   # print('images',images)
    logits_per_image, logits_per_text = model(images, texts)
   # print('logits_per_image, logits_per_text',logits_per_image, logits_per_text)
    probs = logits_per_image.softmax(dim=-1).detach().cpu().numpy()
   # print('probs = logits_per_image',probs)
    index = [i for i in range(batch_size)]
   # print('index',index)
    one_hot = np.zeros((logits_per_image.shape[0], logits_per_image.shape[1]), dtype=np.float32)
   # print('one_hot1',one_hot)
    one_hot[torch.arange(logits_per_image.shape[0]), index] = 1

    one_hot = torch.from_numpy(one_hot).requires_grad_(True)
   # print('one_hot2',one_hot)
    one_hot = torch.sum(one_hot.cuda() * logits_per_image)
   # print('one_hot3',one_hot)
    model.zero_grad()

    image_attn_blocks = list(dict(model.visual.transformer.resblocks.named_children()).values())
    #print('image_attn_blocks',image_attn_blocks)

    if start_layer == -1:
      # calculate index of last layer
      start_layer = len(image_attn_blocks) - 1

    num_tokens = image_attn_blocks[0].attn_probs.shape[-1]
   # print('num_tokens',num_tokens)
    R = torch.eye(num_tokens, num_tokens, dtype=image_attn_blocks[0].attn_probs.dtype).to(device)
   # print('R',R)
    R = R.unsqueeze(0).expand(batch_size, num_tokens, num_tokens)
   # print('R',R)
    for i, blk in enumerate(image_attn_blocks):
        if i < start_layer:
          continue
        grad = torch.autograd.grad(one_hot, [blk.attn_probs], retain_graph=True)[0].detach()
        #print('grad',grad)
        cam = blk.attn_probs.detach()
        #print('cam',cam)
        cam = cam.reshape(-1, cam.shape[-1], cam.shape[-1])
        #print('cam ',cam )
        grad = grad.reshape(-1, grad.shape[-1], grad.shape[-1])
        #print('grad',grad)
        cam = grad * cam
        #print('cam',cam)
        cam = cam.reshape(batch_size, -1, cam.shape[-1], cam.shape[-1])
        #print('cam',cam)
        cam = cam.clamp(min=0).mean(dim=1)
        #print('cam',cam)
        R = R + torch.bmm(cam, R)
        #print('R',R)
    image_relevance = R[:, 0, 1:]
    #print('image_relevance',image_relevance)


    text_attn_blocks = list(dict(model.transformer.resblocks.named_children()).values())

    if start_layer_text == -1:
      # calculate index of last layer
      start_layer_text = len(text_attn_blocks) - 1

    num_tokens = text_attn_blocks[0].attn_probs.shape[-1]
    R_text = torch.eye(num_tokens, num_tokens, dtype=text_attn_blocks[0].attn_probs.dtype).to(device)
    R_text = R_text.unsqueeze(0).expand(batch_size, num_tokens, num_tokens)
    for i, blk in enumerate(text_attn_blocks):
        if i < start_layer_text:
          continue
        grad = torch.autograd.grad(one_hot, [blk.attn_probs], retain_graph=True)[0].detach()
        cam = blk.attn_probs.detach()
        cam = cam.reshape(-1, cam.shape[-1], cam.shape[-1])
        grad = grad.reshape(-1, grad.shape[-1], grad.shape[-1])
        cam = grad * cam
        cam = cam.reshape(batch_size, -1, cam.shape[-1], cam.shape[-1])
        cam = cam.clamp(min=0).mean(dim=1)
        R_text = R_text + torch.bmm(cam, R_text)
    text_relevance = R_text
    

    return text_relevance, image_relevance

In [80]:
def show_image_relevance(image_relevance, image, image_path,orig_image):
    # create heatmap from mask on image
    
    def show_cam_on_image(img, mask):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)

        heatmap = np.float32(heatmap) / 255
        cam = heatmap + np.float32(img)
        cam = cam / np.max(cam)
        return cam

#     fig, axs = plt.subplots(1, 2)
#     axs[0].imshow(orig_image);
#     axs[0].axis('off');
    dim = int(image_relevance.numel() ** 0.5)
    image_relevance = image_relevance.reshape(1, 1, dim, dim)
    image_relevance = torch.nn.functional.interpolate(image_relevance, size=224, mode='bilinear')
    image_relevance = image_relevance.reshape(224, 224).cuda().data.cpu().numpy()
    image_relevance = (image_relevance - image_relevance.min()) / (image_relevance.max() - image_relevance.min())

    binary_mask=(image_relevance>0.07)

    binary_mask_3=np.dstack((binary_mask,binary_mask,binary_mask))
    
    
    image = image[0].permute(1, 2, 0).data.cpu().numpy()
    image = (image - image.min()) / (image.max() - image.min())

    blurred_image=cv2.GaussianBlur(image,(25, 25),0)

    
    o_i=np.where(binary_mask_3,blurred_image,image)
    o_i= cv2.resize(o_i, (224,224))
    out= cv2.resize(o_i, (640,640))
    #for saving
    #######################################################################################
        # Save the modified image and attention heatmap
    filename, ext = os.path.splitext(os.path.basename(image_path))
    modified_filename = filename + ext
    

    output_dir = "/DATA/atul_2221cs20/jitendra/New_model2/output_images_label_02"
    os.makedirs(output_dir, exist_ok=True)

    modified_path = os.path.join(output_dir, modified_filename)
    #print(modified_path)
    if label.item() == 0:
        # Save the original image
        #new 
        print(image_path)
        image_o = np.array(Image.open(image_path))
        print(image_o)
        print(image_o.shape)
        image_o = cv2.cvtColor(image_o, cv2.COLOR_RGB2BGR)
        cv2.imwrite(modified_path, image_o)
        #
        #cv2.imwrite(modified_path, cv2.cvtColor(np.uint8(255 * image), cv2.COLOR_RGB2BGR))
    else:
        # Save the non-misogynous (blurred) image
        cv2.imwrite(modified_path, cv2.cvtColor(np.uint8(255 * out), cv2.COLOR_RGB2BGR))
   

    #cv2.imwrite(modified_path, cv2.cvtColor(np.uint8(255 * o_i), cv2.COLOR_RGB2BGR))
    
    ##########################################################################
    vis = show_cam_on_image(image, image_relevance)

    vis = np.uint8(255 * vis)
    vis = cv2.cvtColor(np.array(vis), cv2.COLOR_RGB2BGR)

#     axs[1].imshow(o_i);
#     axs[1].axis('off');

In [87]:
from CLIP.clip.simple_tokenizer import SimpleTokenizer as _Tokenizer
_tokenizer = _Tokenizer()

def show_heatmap_on_text(text, text_encoding,R_text,image_path,label):
  CLS_idx = text_encoding.argmax(dim=-1)
  R_text = R_text[CLS_idx, 1:CLS_idx]
  #print(R_text)  
    # Calculate a threshold for highlighting (e.g., top 20% most relevant words)
  threshold = np.percentile(R_text.cpu(), 80)

 # Ensure tokens and scores are aligned
  text_tokens_t = _tokenizer.encode(text)
  R_text_list_t = R_text.tolist()  # Convert to list for easier indexing

  highlighted_text = ""
  non_misogynous_text = ""
  for i, (token, score) in enumerate(zip(text_tokens_t, R_text_list_t)):
    word = _tokenizer.decode([token])  # Decode individual token
    if score > threshold:
      highlighted_text += f"{word}"
    else:
      non_misogynous_text += f"{word} "
  
  #print("highlighted_text",highlighted_text)
  #print("non_misogynous_text",non_misogynous_text)
  # Save non_misogynous_text to a CSV file
  csv_filename = "/DATA/atul_2221cs20/jitendra/New_model2/full_text_description_01.csv"
  file_exists = os.path.isfile(csv_filename)
  filename, ext = os.path.splitext(os.path.basename(image_path))
  modified_filename = filename + ext
      # Determine the text to save based on the label
  non_misogynous_text = "" if label.item() == 0 else non_misogynous_text.strip() 
  misogynous_text = "" if label.item() == 0 else highlighted_text.strip() 
  text= text
  with open(csv_filename, mode='a', newline='') as file:
      writer = csv.writer(file)
      if not file_exists:
          writer.writerow(['name','label', 'non_misogynous_text','misogynous_text','text'])
            
      writer.writerow([modified_filename,label.item(), non_misogynous_text,misogynous_text,text])
    
  text_scores = R_text / R_text.sum()
  text_scores = text_scores.flatten()
  #print(text_scores)
  text_tokens=_tokenizer.encode(text)
  #print('text_tokens',text_tokens)  
  text_tokens_decoded=[_tokenizer.decode([a]) for a in text_tokens]
  #print(text_tokens_decoded)
  vis_data_records = [visualization.VisualizationDataRecord(text_scores,0,0,0,0,0,text_tokens_decoded,1)]
  
  #print(visualization.visualize_text(vis_data_records))
  visualization.visualize_text(vis_data_records)

In [88]:
clip.clip._MODELS = {
    "ViT-B/32": "https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt",
    
    
}

In [89]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("/DATA/atul_2221cs20/jitendra/New_model2/checkpoints/last_model2.pt", device=device, jit=False)

In [90]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [91]:
# import csv
# img_path = "/DATA/atul_2221cs20/jitendra/New_model2/archive/MAMI_2022_images/training_images/5906.jpg"
# print(img_path)
# img = preprocess(Image.open(img_path)).unsqueeze(0).to(device)

# texts = ["Ohh Shit!!!! It's Cameron Diaz!! loppusem"]
# text = clip.tokenize(texts).to(device)
# label=1
# R_text, R_image = interpret(model=model, image=img, texts=text, device=device)
# batch_size = text.shape[0]
# for i in range(batch_size):
#   #print(R_text[i])  
#   print(img_path)
#   show_heatmap_on_text(texts[i], text[i], R_text[i],image_path=img_path,label=label)
#   show_image_relevance( R_image[i], img, image_path=img_path, orig_image=Image.open(img_path))
#   plt.show()

In [92]:
import os
import csv
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from transformers import BertTokenizer



# Define the paths to your dataset folder and CSV file
dataset_folder = "/DATA/atul_2221cs20/jitendra/New_model2/archive/MAMI_2022_images/test_images"
csv_file = "/DATA/atul_2221cs20/jitendra/New_model2/archive/MAMI_test_set01.csv"

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)



# Define a custom dataset class for MAMI dataset
class MAMIDataset(Dataset):
    def __init__(self, csv_file, root_dir, preprocess):
        self.preprocess = preprocess
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = self.preprocess(Image.open(img_path))
        label = self.data.iloc[idx, 1]  # Assuming label is in the second column
        text = self.data.iloc[idx, 2][:77]  # Assuming text is in the third column
        return image, text, label, img_path

dataset = MAMIDataset(csv_file, dataset_folder, preprocess)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [93]:
for i, (image, texts, label, img_path) in enumerate(dataloader):
    # print(image, texts, label, img_path)


#     show_heatmap_on_text(text[0], text_encodings, R_texts, image_path=img_path[0])

#     show_image_relevance(R_image, image, image_path=img_path[0], orig_image=Image.open(img_path[0]))
#     plt.show()
     img_path = img_path[0]
     #label=label[0]
     #print(label)
    
     
     img = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
     print()
     text = clip.tokenize([texts[0]]).to(device)
     #text = clip.tokenize(text).to(device)


     R_text, R_image = interpret(model=model, image=img, texts=text, device=device)
     batch_size = text.shape[0]
     #print(batch_size)   
     for i in range(batch_size):
          #print(R_text[i])  
         #print(img_path)
         #print(text)  
         show_heatmap_on_text(texts[i], text[i], R_text[i],image_path=img_path,label=label)
         show_image_relevance( R_image[i], img, image_path=img_path, orig_image=Image.open(img_path))
         plt.show()

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,0.00,g his . un dying fidelity steve is hot and perfect in every way . instagram his .


/DATA/atul_2221cs20/jitendra/New_model2/archive/MAMI_2022_images/test_images/15001.jpg
[[[25 25 37]
  [25 25 37]
  [21 23 35]
  ...
  [15 15 17]
  [15 15 17]
  [15 15 17]]

 [[24 24 36]
  [27 27 39]
  [22 24 36]
  ...
  [15 15 17]
  [15 15 17]
  [15 15 17]]

 [[23 25 37]
  [26 28 40]
  [23 25 38]
  ...
  [15 15 17]
  [15 15 17]
  [15 15 17]]

 ...

 [[21 28 46]
  [26 33 51]
  [26 32 54]
  ...
  [19 24 44]
  [22 27 49]
  [16 21 43]]

 [[27 36 53]
  [30 39 56]
  [28 34 56]
  ...
  [19 25 47]
  [24 28 53]
  [24 28 53]]

 [[29 38 55]
  [26 35 52]
  [28 34 56]
  ...
  [20 26 48]
  [17 21 48]
  [25 29 56]]]
(640, 640, 3)



True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,0.00,how li mag ined myself as a teacher ...... how i feel as a teacher ... panda cont


/DATA/atul_2221cs20/jitendra/New_model2/archive/MAMI_2022_images/test_images/15002.jpg
[[[204 207 212]
  [197 201 204]
  [191 192 196]
  ...
  [143 106  51]
  [145 108  53]
  [146 111  55]]

 [[205 208 213]
  [195 199 202]
  [190 191 193]
  ...
  [139 102  47]
  [141 106  50]
  [142 107  51]]

 [[206 207 211]
  [196 197 199]
  [190 190 188]
  ...
  [138 103  49]
  [138 105  51]
  [140 107  53]]

 ...

 [[221 192 150]
  [218 189 147]
  [205 176 134]
  ...
  [191 168 116]
  [206 183 131]
  [212 189 137]]

 [[192 163 119]
  [157 128  84]
  [133 101  60]
  ...
  [210 189 136]
  [196 175 122]
  [179 158 105]]

 [[186 157 113]
  [151 122  78]
  [159 127  86]
  ...
  [211 190 137]
  [198 177 124]
  [196 176 123]]]
(640, 589, 3)



True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,0.00,where will you be when diar rhea strike meme center . com m ame center


...

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,0.00,looks like the air bags deployed 1 2 3 4 4 9 8


/DATA/atul_2221cs20/jitendra/New_model2/archive/MAMI_2022_images/test_images/17080.jpg
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [ 24  26  13]
  [ 27  29  16]
  [ 26  28  15]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [ 25  28  17]
  [ 28  31  20]
  [ 29  32  21]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [ 22  28  16]
  [ 29  32  23]
  [ 31  34  25]]]
(366, 640, 3)



True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,0.00,half woman half horse


/DATA/atul_2221cs20/jitendra/New_model2/archive/MAMI_2022_images/test_images/17081.jpg
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [247 247 247]
  [247 247 247]
  [247 247 247]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [247 247 247]
  [247 247 247]
  [247 247 247]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [247 247 247]
  [247 247 247]
  [247 247 247]]]
(640, 586, 3)



True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,0.00,we don 't mind if a man tries to rape you . g & p we only mind if you don 't carry


In [ ]:
!ls

In [ ]:
!ls

In [ ]:
import numpy as np
import torch,os
from pkg_resources import packaging
import clip
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from PIL import Image

BATCH_SIZE = 100
EPOCH = 5

print("Torch version:", torch.__version__)
print(torch.cuda.is_available())
print(clip.available_models())
# If using GPU then use mixed precision training.
device = "cuda:0" if torch.cuda.is_available() else "cpu" 

#Must set jit=False for training
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) 

#checkpoint = torch.load("./ViT-B-32.pt")
#model.load_state_dict(checkpoint['model_state_dict'])

#Define Function
class image_title_dataset():
    def __init__(self, list_image_path,list_txt):
        self.list_txt = list_txt
        self.image_path = list_image_path
        self.title  = clip.tokenize(list_txt)
        #you can tokenize everything at once in here(slow at the beginning), or tokenize it in the training loop.

    def __len__(self):
        return len(self.list_txt)

    def __getitem__(self, idx):
        image = preprocess(Image.open(self.image_path[idx])) # Image from PIL module
        title = self.title[idx]
        return image,title


# use my dataset
list_image_path = []
list_txt = []
Path = "./image/"
list_dir = os.listdir(Path)
for subdir in list_dir:
    Path_subdir = Path + subdir + '/'
    list_file= os.listdir(Path_subdir)
    for i in range(len(list_file)):
        list_image_path.append(Path_subdir + list_file[i])
        list_txt.append(subdir)



print("len[list_image_path] = ",len(list_image_path))    
print("len[list_txt] = ",list_txt[0],len(list_txt))

dataset = image_title_dataset(list_image_path,list_txt)
train_dataloader = DataLoader(dataset,batch_size = BATCH_SIZE) #Define your own dataloader

#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

if device == "cpu":
  model.float()
else :
  clip.model.convert_weights(model) # Actually this line is unnecessary since clip by default already on float16

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

# add your own code to track the training progress.


for epoch in range(EPOCH):

  for batch in train_dataloader :
      optimizer.zero_grad()
      images,texts = batch 
    
      images= images.to(device)
      texts = texts.to(device)
    
      logits_per_image, logits_per_text = model(images, texts)

      ground_truth = torch.arange(len(images),dtype=torch.long,device=device)

      total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
      total_loss.backward()

      if device == "cpu":
         optimizer.step()
      else : 
        convert_models_to_fp32(model)
        optimizer.step()
        clip.model.convert_weights(model)
  if (epoch%100==0):
      print("[",epoch,"]\t total_loss = ", total_loss)

torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, f"model_checkpoint/model_clip_test.pt")
        #just change to your preferred folder/filename

In [ ]:
#noiw 

In [ ]:
# Latest Update : 18 July 2022, 09:55 GMT+7

# TO ADD :
# Gradient Checkpointing
# Filter out bias from weight decay
# Decaying learning rate with cosine schedule
# Half-precision Adam statistics
# Half-precision stochastically rounded text encoder weights were used

#BATCH_SIZE must larger than 1

device = "cuda:0" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training

class image_title_dataset(Dataset):
    def __init__(self, list_image_path,list_txt):

        self.image_path = list_image_path
        self.title  = clip.tokenize(list_txt) #you can tokenize everything at once in here(slow at the beginning), or tokenize it in the training loop.

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        image = preprocess(Image.open(self.image_path[idx])) # Image from PIL module
        title = self.title[idx]
        return image,title

# use your own data
list_image_path = ['folder/image1.jpg','folder2/image2.jpg'] 
list_txt = ['description for image1.jpg' , 'description for image2.jpg']
dataset = image_title_dataset(list_image_path,list_txt)
train_dataloader = DataLoader(dataset,batch_size = BATCH_SIZE) #Define your own dataloader

#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 


if device == "cpu":
  model.float()
else :
  clip.model.convert_weights(model) # Actually this line is unnecessary since clip by default already on float16

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

# add your own code to track the training progress.
for epoch in range(EPOCH):
  for batch in train_dataloader :
      optimizer.zero_grad()

      images,texts = batch 
    
      images= images.to(device)
      texts = texts.to(device)
    
      logits_per_image, logits_per_text = model(images, texts)

      ground_truth = torch.arange(len(images),dtype=torch.long,device=device)

      total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
      total_loss.backward()
      if device == "cpu":
         optimizer.step()
      else : 
        convert_models_to_fp32(model)
        optimizer.step()
        clip.model.convert_weights(model)

torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, f"model_checkpoint/model_10.pt") #just change to your preferred folder/filename


        

In [ ]:
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
checkpoint = torch.load("model_checkpoint/model_10.pt")

# Use these 3 lines if you use default model setting(not training setting) of the clip. For example, if you set context_length to 100 since your string is very long during training, then assign 100 to checkpoint['model_state_dict']["context_length"] 
checkpoint['model_state_dict']["input_resolution"] = model.input_resolution #default is 224
checkpoint['model_state_dict']["context_length"] = model.context_length # default is 77
checkpoint['model_state_dict']["vocab_size"] = model.vocab_size 

model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
#fine-tuning clip model